In [ ]:
#train haro az folder avalie(images1) bekhone va be 


In [2]:
import os
import shutil
import random

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns



2022-02-19 16:29:22.237726: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-19 16:29:22.237764: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
path = '/home/amindo/Desktop/javaneh_amlak/' 
#action_logs =  pd.read_parquet(path + 'action_logs.parquet')
#ladder =  pd.read_parquet(path + 'ladder.parquet')
posts =  pd.read_parquet(path + 'posts.parquet')
train_labels =  pd.read_csv(path + 'train_labels.csv')
validation =  pd.read_csv(path + 'validation.csv')

In [3]:
df = posts.join(train_labels.set_index('token'), on='token')
df = df.loc[(df.result.isnull()==False)&(df.image_count>0)]
df = df.sort_values(by = 'token').reset_index()
df_ok = df.loc[df.result == 'ok']
df_fake = df.loc[df.result == 'fake-post']
train_ok_list = list(df_ok.token)
train_fake_list = list(df_fake.token)

In [6]:
#remove_train_error_imgae
def remove_image_train(path):
    path1 = path + 'images1'
    for i in os.listdir(path1):
        path2 = path1 +'/'+ i + '/'
        for img in os.listdir(path2): 
            try:
                path3 = path2 + img
                image = cv2.imread(path3)
                img_arr = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            except Exception as e:
                os.remove(path3)

In [7]:
remove_image_train(path)

In [8]:
# build train and val image data set
def transfe_ok_fake(path,ok_list):
    list_ok = []
    list_fake = []
    path1 = path + 'images1/'
    list1 = os.listdir(path1)    
    for i in list1:
        path2 = path1 + i + '/'
        list2 = os.listdir(path2)
        for j in list2:
            name = i + '/' + j
            if i in ok_list:
                sor = path2 + j
                name = i + '@' + j
                des = path + 'ok' + '/' + name
                shutil.copy(sor,des)
            else:
                sor = path2 + j
                name = i + '@' + j
                des = path + 'fake' + '/' + name
                shutil.copy(sor,des)          

In [11]:
transfe_ok_fake(path,train_ok_list)

In [20]:
def final_transfer_train_val(path):
    list_ok = os.listdir(path + '/ok' )
    ok_val_num =int(len(list_ok)/5)
    list_fake = os.listdir(path + '/fake')
    fake_val_num = int(len(list_fake)/5)
    ok_sample = random.sample(list_ok,ok_val_num)
    fake_sample = random.sample(list_fake,fake_val_num)
    des1 = path + '/' + 'images_sec/'+ 'validation/' 
    for ok in list_ok:
        sor = path + 'ok/' + ok 
        des_val = des1 + 'ok'
        des_train = path + '/' + 'images_sec/train/ok'
        if ok in ok_sample:
            shutil.copy(sor,des_val)
        else:
            shutil.copy(sor,des_train)
    for fake in list_fake:
        sor = path + 'fake/' + fake 
        des_val = des1 + 'fake'
        des_train = path + '/' + 'images_sec/train/fake'
        if fake in fake_sample:
            shutil.copy(sor,des_val)
        else:
            shutil.copy(sor,des_train)
            
            

In [21]:
final_transfer_train_val(path)

In [26]:
def transfer_just_image_issue(list_image_tag_issue):
    path1 = path + 'images1/'
    for token in list_image_tag_issue:
        path2 = path1 + token
        list1 = os.listdir(path2)
        for i in list1:
            sor = path2 + '/' + i
            name = token + '@' + i
            des = path + 'tag_image_issue' + '/' + name
            shutil.copy(sor,des)

In [30]:
df = posts.join(train_labels.set_index('token'), on='token')
df = df.loc[(df.result.isnull()==False)&((df.tag == 'عکس آگهی با اطلاعات نوشته شده مطابقت ندارد')|(df.tag =='عکس با آگهی مطابقت نداشت' ))&(df.image_count >0)]

In [29]:
df.loc[(df.result.isnull()==False)&((df.tag == 'عکس آگهی با اطلاعات نوشته شده مطابقت ندارد')|(df.tag =='عکس با آگهی مطابقت نداشت' ))&(df.image_count == 0)]

,token,first_published_at,data,image_count,device_id,username,retiredAtUtc,city,district,category,user_hash,call_date,result,tag
124225,QYozAHeP,2022-01-19 16:34:57.090,"{""rent"": {""mode"": ""مقطوع"", ""value"": 16000000},...",0,FcVF1UQsRieZH4fkRQ2g0w,real-estate-business_agent_149905,None,tehran,niavaran,apartment-rent,28f8916ba61d1c6ae68f,2022-01-25,fake-post,عکس آگهی با اطلاعات نوشته شده مطابقت ندارد
125256,QYLW4lnU,2021-11-10 12:50:23.397,"{""rent"": {""mode"": ""مجانی"", ""value"": 0}, ""size""...",0,t6FfyaKNQg27S0qbo5L2XQ,real-estate-business_3944,None,tehran,gholhak,apartment-rent,9607ce3d2e7ffc51d284,2021-11-12,fake-post,عکس با آگهی مطابقت نداشت
127836,QYluy9FY,2021-11-29 23:10:07.162,"{""rent"": {""mode"": ""مقطوع"", ""value"": 9000000}, ...",0,GZjMKHFrTDiZFsbr-ypBAg,None,None,tehran,fasham,apartment-rent,220e5cc161df2148bf00,2021-12-07,fake-post,عکس با آگهی مطابقت نداشت
129188,QYv-3p17,2021-12-07 19:20:10.174,"{""rent"": {""mode"": ""مجانی"", ""value"": 0}, ""size""...",0,QsH4ODJJQE6QWPS7DTPMdQ,None,None,tehran,shadabad,apartment-rent,f022a5b82f3d1c5fa538,2021-12-08,fake-post,عکس با آگهی مطابقت نداشت


In [49]:
#make new data set for tag_image issue
#moshkel dara
df = posts.join(train_labels.set_index('token'), on='token')
df = df.loc[(df.result.isnull()==False)&((df.tag == 'عکس آگهی با اطلاعات نوشته شده مطابقت ندارد')|(df.tag =='عکس با آگهی مطابقت نداشت' ))&(df.image_count >0)]
list_image_tag_fake = list(df.token)
#kharab ha
list_kharab = list(df.loc[(df.result.isnull()==False)&((df.tag == 'عکس آگهی با اطلاعات نوشته شده مطابقت ندارد')|(df.tag =='عکس با آگهی مطابقت نداشت' ))&(df.image_count ==0)].token)
# ok ha 
list_image_tag_ok = os.listdir(path + 'images1')
[list_image_tag_ok.remove(i) for i in list_image_tag_issue if i in list_image_tag_issue]
[list_image_tag_ok.remove(i) for i in list_kharab if i in list_kharab]


[]

In [52]:
def tag_transfe_ok_fake(path,list_image_tag_ok):
    list_ok = []
    list_fake = []
    path1 = path + 'images1/'
    list1 = os.listdir(path1)    
    for i in list1:
        path2 = path1 + i + '/'
        list2 = os.listdir(path2)
        for j in list2:
            name = i + '/' + j
            if i in list_image_tag_ok:
                sor = path2 + j
                name = i + '@' + j
                des = path + 'ok_image_tag_issue' + '/' + name
                shutil.copy(sor,des)
            else:
                sor = path2 + j
                name = i + '@' + j
                des = path + 'fake_image_tag_issue' + '/' + name
                shutil.copy(sor,des)

In [53]:
tag_transfe_ok_fake(path,list_image_tag_ok)

In [56]:
path

'/home/amindo/Desktop/javaneh_amlak/'

In [57]:
def image_tag_final_transfer_train_val(path):
    list_ok = os.listdir(path + '/ok_image_tag_issue' )
    ok_val_num =int(len(list_ok)/5)
    list_fake = os.listdir(path + '/fake_image_tag_issue')
    fake_val_num = int(len(list_fake)/5)
    ok_sample = random.sample(list_ok,ok_val_num)
    fake_sample = random.sample(list_fake,fake_val_num)
    des1 = path + '/' + 'tag_image_issue/'+ 'validation/' 
    for ok in list_ok:
        sor = path + 'ok_image_tag_issue/' + ok 
        des_val = des1 + 'ok'
        des_train = path + '/' + 'tag_image_issue/train/ok'
        if ok in ok_sample:
            shutil.copy(sor,des_val)
        else:
            shutil.copy(sor,des_train)
    for fake in list_fake:
        sor = path + 'fake_image_tag_issue/' + fake 
        des_val = des1 + 'fake'
        des_train = path + '/' + 'tag_image_issue/train/fake'
        if fake in fake_sample:
            shutil.copy(sor,des_val)
        else:
            shutil.copy(sor,des_train)
            
         

In [2]:
# drop half of train ok 
list1 = os.listdir('/home/amindo/Desktop/javaneh_amlak/tag_image_issue/train/0')
drop_list = random.sample(list1,int(len(list1)/2))

In [3]:
for i in list1:
    if i in drop_list:
        
        shutil.move(,)

['QYxGMVhT@2.jpg',
 'QYnzscVI@3.jpg',
 'QYSO3zIR@3.jpg',
 'QYNGG5lJ@1.jpg',
 'AYnure89@4.jpg',
 'QYCuz3__@0.jpg',
 'QYLSTAnd@0.jpg',
 'QYauYvU_@7.jpg',
 'QYnyDspn@2.jpg',
 'QYTqw-ER@2.jpg',
 'QYdWPiRe@0.jpg',
 'QYKuhGnf@1.jpg',
 'QYaDOZ6P@1.jpg',
 'QYQXgR7C@3.jpg',
 'QYXK4oBb@12.jpg',
 'QY5VHeh9@0.jpg',
 'QYa3u7wu@8.jpg',
 'QYQqsu2Z@1.jpg',
 'QYKeH3BU@13.jpg',
 'QY2tU8zB@10.jpg',
 'QYbeTX97@2.jpg',
 'QYlxU6-Y@0.jpg',
 'QYpBrOl7@4.jpg',
 'QYhAApr7@8.jpg',
 'QY9a7cbm@1.jpg',
 'QYuVNU1L@2.jpg',
 'QYSUVenl@1.jpg',
 'QYR-Psc7@2.jpg',
 'QYGCd7fP@2.jpg',
 'QYEqOjLA@19.jpg',
 'QYoOMtTk@7.jpg',
 'QYt6fsxV@2.jpg',
 'QYeCUmbn@3.jpg',
 'QY1yK8o7@2.jpg',
 'QYMitAtH@1.jpg',
 'QYXu7AFR@0.jpg',
 'QYhiFTKN@2.jpg',
 'QYTiFhHQ@0.jpg',
 'QYVmiXRW@4.jpg',
 'QYcDgNzG@0.jpg',
 'QY0dE3PH@8.jpg',
 'QYqbYFrV@0.jpg',
 'QYkO_tOB@4.jpg',
 'QY11pgrx@6.jpg',
 'QYGOVjuP@3.jpg',
 'QYYCg7Ik@11.jpg',
 'QYB-hTS5@2.jpg',
 'QYKayFnj@2.jpg',
 'QYZaiLkA@2.jpg',
 'QYRuQq52@0.jpg',
 'QYR-Psc7@4.jpg',
 'QYLKWUPM@1.jpg',
 'QYHec

In [ ]:
#drop half of val ok

In [3]:
list1 = os.listdir('/home/amindo/Desktop/javaneh_amlak/tag_image_issue/validation/0')
drop_list = random.sample(list1,int(len(list1)/2))

In [4]:
for i in list1:
    if i in drop_list:
        sor = '/home/amindo/Desktop/javaneh_amlak/tag_image_issue/validation/0/' + i
        des = '/home/amindo/Desktop/javaneh_amlak/drop_val_ok_image/'
        shutil.move(sor,des)

In [ ]:
def remove_image_train(path):
    path1 = path + 'images'
    for i in os.listdir(path1):
        path2 = path1 +'/'+ i + '/'
        for img in os.listdir(path2): 
            try:
                path3 = path2 + img
                image = cv2.imread(path3)
                img_arr = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                name = i + +@ + img
                des = path + '/' + 'all_images' + '/' + name
                shutil.move(path3,des)
            except Exception as e:
                os.remove(path3)

In [7]:
path ='/home/amindo/Desktop/javaneh_amlak/'

In [15]:
def transfe_test_all(path):
    list_ok = []
    list_fake = []
    path1 = path + 'images_sec/test/'
    list1 = os.listdir(path1)    
    for i in list1:
        path2 = path1 + i + '/'
        list2 = os.listdir(path2)
        for j in list2:
            sor = path2 + j
            name = i + '@' + j
            des = path + 'images_sec/test_all' + '/' + name
            shutil.copy(sor,des)

In [16]:
transfe_test_all(path)